## **演示1001：K近邻分类**

### **提出问题**
已知N维空间中若干个点的坐标，以及这些点所属的类别(子空间)。给定新的点坐标，如何判断该点应被划入哪个类别(子空间)？  
![](../images/100101.png)

### **分析问题**
* KNN算法基本思想：
已知一批数据集及其对应的分类标签，输入测试数据，将测试数据的特征与训练集中对应的特征进行相互比较，找到训练集中与之最为相似的前K个数据，则该测试数据对应的类别就是K个数据中出现次数最多的那个分类。具体步骤：  
(1) 计算测试数据与各个训练数据之间的距离；  
(2) 按照距离的递增关系进行排序；  
(3) 选取距离最小的K个点；  
(4) 确定前K个点所在类别的出现频率；  
(5) 返回前K个点中出现频率最高的类别作为测试数据的预测分类。  
与之前线性回归、逻辑回归和朴素贝叶斯模型不同，KNN算法不会形成假设函数。每次预测时，必须即时跟所有训练数据进行计算，因此工作量很大。

* K值的选取：
 * K可以视为一个hyper-parameter(超参数)，一般需要通过交叉验证的方法来选取最优值
 * 如果K值太小就意味着整体模型变得复杂，容易发生过拟合(High Variance)，即如果邻近的实例点恰巧是噪声，预测就会出错，极端的情况是K=1，称为最近邻算法，对于待预测点x，与x最近的点决定了x的类别
 * K值的增大意味着整体的模型变得简单，极端的情况是K=N，那么无论输入实例是什么，都简单地预测它属于训练集中最多的类。这样的模型过于简单，容易发生欠拟合(High Bias)

* K-D树方法：
 * K近邻法的最简单实现是线性扫描（又称暴力法），这时要计算输入实例与每一个训练实例的距离，当训练集很大时，计算非常耗时，这种方法是不可行的
 * 为了提高K近邻搜索的效率，可以考虑使用特殊的结构存储训练数据，以减少计算距离的次数。K-D树提供了一种最基本的方法
 * 可将K-D树看成类似于二叉查找树的数据结构，每个训练数据存储在这个数据结构中。在查找数据时，采用类似于中序遍历的算法。（但实际过程要复杂许多）。
 * K-D树搜索的平均复杂度为$O(logM)$。其中,M为样本数量。但是当Feature数量N很大时，搜索性能将急剧下降。一般K-D树适于M>>N的情形

* KNN的优缺点：
 * 精度较高，对异常值不太敏感
 * 特别适合多分类的情况
 * 简单易实现
 * 很多情况下比朴素贝叶斯的效果更好
 * 计算复杂度高，空间复杂度高

### **案例1：实现简单的KNN算法**
> 给定若干组数据及其对应分类，再给定一组测试数据，使用KNN计算出该测试数据最接近的分类

In [5]:
''' 手工实现KNN算法进行简单的数据分类 '''

import numpy as np

# 给出训练数据以及对应的类别
def createDataSet():
    group = np.array([[1.0, 2.0], [1.2, 0.1], [0.1, 1.4], [0.3, 3.5]])
    labels = ['A', 'B', 'C', 'D']
    return group,labels

# 通过KNN进行分类
def classify(input, dataSet, label, k):
    dataSize = dataSet.shape[0]
    # 计算欧式距离
    # tile将input按照(dataSize, 1)的shape进行复制，使之与dataSet的shape一致，从而可以对应元素相减
    diff = np.tile(input, (dataSize, 1)) - dataSet
    sqdiff = diff ** 2
    squareDist = np.sum(sqdiff, axis=1)             # 行向量分别相加，从而得到新的一个行向量
    dist = squareDist ** 0.5
    
    # argsort()根据元素的值从大到小对元素进行排序，返回下标
    sortedDistIndex = np.argsort(dist)              

    classCount = {}
    for i in range(k):
        voteLabel = label[sortedDistIndex[i]]
        # 对选取的K个样本所属的类别个数进行统计
        # classCount.get(voteLabel, 0)：如果voteLabel在classCount已经存在，则返回它对应值
        # 如果不存在，则返回默认值0
        classCount[voteLabel] = classCount.get(voteLabel, 0) + 1
    
    # 选取出现的类别次数最多的类别
    maxCount = 0
    for key,value in classCount.items():
        if value > maxCount:
            maxCount = value
            classes = key

    return classes   


dataSet, labels = createDataSet()
input = np.array([1.1, 0.3])
K = 3
output = classify(input, dataSet, labels, K)
print("测试数据为:",input,"分类结果为：",output)

测试数据为: [1.1 0.3] 分类结果为： B


### **案例2：使用sklearn.neighbors.NearestNeighbors**
* NearestNeightbors能够直接获得最接近的K个分类(以该分类对应的下标索引形式返回)，以及对应的距离
* 选择KNN的实现算法(指定algorithm参数)
 * 'brute'：暴力法
 * 'kd-tree'
 * 'ball-tree'
* 设置权重模式(指定weights参数)
 * 'uniform'：所有点的权重相同
 * 'distance'：越接近的点，权重越大
 * [callable]：允许通过一个自定义的函数来确定各个点的权重
* 设置距离计算模式(指定metric参数)
 * 默认为'minkowski'。其它可选如：euclidean,manhattan,chebyshev等

In [4]:
''' 使用NearestNeighbors进行分类 '''

import numpy as np
from sklearn.neighbors import NearestNeighbors

# 给出训练数据以及对应的类别
def createDataSet():
    group = np.array([[1.0, 2.0], [1.2, 0.1], [0.1, 1.4], [0.3, 3.5]])
    labels = ['A', 'B', 'C', 'D']
    return group,labels

dataSet, labels = createDataSet()
K = 3
model = NearestNeighbors(n_neighbors=K, algorithm='ball_tree')
model.fit(dataSet)
input = np.array([[1.1, 0.3]])
distances, indices = model.kneighbors(input)
print("测试数据为:",input)
print("测试数据返回的索引集合尺寸：", indices.shape)
print("分类结果，由近及远依次为：",labels[indices[0,0]],labels[indices[0,1]],labels[indices[0,2]])

测试数据为: [[1.1 0.3]]
测试数据返回的索引集合尺寸： (1, 3)
分类结果，由近及远依次为： B C A


### **案例3：使用sklearn.neighbors.NearestNeighbors识别手写数字图片**

In [3]:
import numpy as np
import collections as col
from sklearn.neighbors import NearestNeighbors

trainData = np.loadtxt(open('digits_training.csv', 'r'), delimiter=",",skiprows=1)
MTrain, NTrain = np.shape(trainData)
xTrain = trainData[:, 1:NTrain]
yTrain = trainData[:, 0]
print("装载训练数据：", MTrain, "条，训练中......")

K = 3
model = NearestNeighbors(n_neighbors=K, algorithm='auto')
model.fit(xTrain)
print("训练完毕")

testData = np.loadtxt(open('digits_testing.csv', 'r'), delimiter=",",skiprows=1)
MTest,NTest = np.shape(testData)
xTest = testData[:, 1:NTest]
yTest = testData[:, 0]
print("装载测试数据：", MTest, "条，预测中......")

indices = model.kneighbors(xTest, return_distance=False)
print("测试数据返回的索引矩阵尺寸：", indices.shape)
yPredicts = np.zeros(MTest)
for i in np.arange(0, MTest):
    counter = col.Counter(indices[i])               # 统计最近的K个索引分别出现的次数
    max_index = counter.most_common()[0][0]         # 获取次数最多的那个索引值
    yPredicts[i] = yTrain[max_index]

errors = np.count_nonzero(yTest - yPredicts)
print("预测完毕。错误：", errors, "条")
print("测试数据正确率:", (MTest - errors) / MTest)   # 约0.944的正确率

装载训练数据： 5000 条，训练中......
训练完毕
装载测试数据： 500 条，预测中......
测试数据返回的索引矩阵尺寸： (500, 3)
预测完毕。错误： 28 条
测试数据正确率: 0.944
